In [0]:
import pandas as pd
import numpy as np

from scipy.optimize import minimize

### Definición de vectores

In [0]:
# se inicializa la matriz de flujos, frecuencias con los indices (nombres de las ciudades)
matr = pd.DataFrame(index=['Aguascalientes', 'CDMX', 'Chihuahua', 'Guadalajara', 'Monterrey', 'Morelia', 'Puebla', 'Querétaro', 'Saltillo', 'Toluca', 'Nuevo Laredo'])  # nuevo laredo como estados unidos
matr

""
Aguascalientes
CDMX
Chihuahua
Guadalajara
Monterrey
Morelia
Puebla
Querétaro
Saltillo
Toluca


In [0]:
# se busca latitud y longitud de cada una
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="foursquare_agent")
matr['lat'] = np.nan
matr['lon'] = np.nan

for city in matr.index:
  location = geolocator.geocode(city)
  matr.loc[matr.index == city, 'lat'] = location.latitude
  matr.loc[matr.index == city, 'lon'] = location.longitude
matr


,lat,lon
Aguascalientes,22.000000,-102.500000
CDMX,19.320556,-99.151701
Chihuahua,28.500000,-106.000000
Guadalajara,20.672037,-103.338396
Monterrey,25.639784,-100.293102
Morelia,19.702712,-101.192382
Puebla,18.833333,-98.000000
Querétaro,20.854257,-99.847560
Saltillo,25.423043,-100.992751
Toluca,19.292545,-99.656901


In [0]:
import folium # https://python-visualization.github.io/folium/quickstart.html
m = folium.Map(location=[geolocator.geocode('Mexico').latitude, geolocator.geocode('Mexico').longitude ], zoom_start=6)
for i in range(len(matr.index)):
  folium.Marker([matr.iloc[i, 0], matr.iloc[i, 1]]).add_to(m)
m

In [0]:
# completar datos segun instrucciones del proyecto

# cuando no hay un valor, se reemplaza con 0
vector = {
    'freq_compra': [0, .40, 0, .25, .10, 0, .05, .10, 0, 0, .10],
    'pct_compra_monetario': [0, .35, 0, .25, .10, 0, .05, .05, 0, 0, .20],
    'freq_venta': [.02, .21, .05, .15, .11, .06, .09, .11, .02, .03, .06 ], # otros .09
    'pct_utilidades': [.01, .24, .03, .19, .14, .03, .07, .11, .01, .01, .09]  # otros .7
}
# nota: a los otros no se los puede incluir ya que no tienen coordenadas para asociarlos
# por lo tanto se los transforma, dividiendo a cada valor sobre el total que suman
# esto permitirá que el vector de penalización visto más adelante no esté sesgado
suma_venta = sum(vector['freq_venta'])
vector['freq_venta'] = [valor / suma_venta  for valor in vector['freq_venta']]
suma_utilidad = sum(vector['pct_utilidades'])
vector['pct_utilidades'] = [valor / suma_utilidad for valor in vector['pct_utilidades']]

# comprobar
for k in vector.keys():
  print(f'Suma de {k} es ',  sum(vector[k]))

vector = pd.DataFrame(vector, index=matr.index)


matr = pd.merge(matr, vector, left_index=True, right_index=True)
matr

Suma de freq_compra es  1.0
Suma de pct_compra_monetario es  1.0
Suma de freq_venta es  1.0
Suma de pct_utilidades es  0.9999999999999998


,lat,lon,freq_compra,pct_compra_monetario,freq_venta,pct_utilidades
Aguascalientes,22.000000,-102.500000,0.00,0.00,0.021978,0.010753
CDMX,19.320556,-99.151701,0.40,0.35,0.230769,0.258065
Chihuahua,28.500000,-106.000000,0.00,0.00,0.054945,0.032258
Guadalajara,20.672037,-103.338396,0.25,0.25,0.164835,0.204301
Monterrey,25.639784,-100.293102,0.10,0.10,0.120879,0.150538
Morelia,19.702712,-101.192382,0.00,0.00,0.065934,0.032258
Puebla,18.833333,-98.000000,0.05,0.05,0.098901,0.075269
Querétaro,20.854257,-99.847560,0.10,0.05,0.120879,0.118280
Saltillo,25.423043,-100.992751,0.00,0.00,0.021978,0.010753
Toluca,19.292545,-99.656901,0.00,0.00,0.032967,0.010753


### Determinación del vector de 'penalización'
 Cuando se recorre una distancia, debe ser multiplicada también por un vector que representa el costo total, teniendo en cuenta flujos y frecuencias, tanto para clientes como para proveedores. <br/>
 Por esto, para clientes y proveedores por separado se hace una suma de la frequencia de compra (o venta) y del porcentaje de compra(o venta) monetario. <br/>
 Se hace una suma porque no se sabe el margen de ventas / compras que se tiene. <br/>
   Por último, a la suma de los clientes se la promedia de forma ponderada con la de los proveedores, según las instrucciones, según lo siguiente: <br/>
   `Por último, para determinar que peso se le debe dar a la ubicación de los proveedores y que peso a la ubicación
de los clientes, se sabe que el costo asociado al trasporte de un producto de proveedor a fabrica representa sólo
el 40% del costo asociado al costo de transporte de fabrica a clientes.`


In [0]:
matr['suma_provs'] = matr['freq_compra'] + matr['pct_compra_monetario']
matr['suma_clientes'] = matr['freq_venta'] +  1 - matr['pct_utilidades']  # se saca el complemento de utilidades, tal que se penalice por menos utilidades
matr['penal'] = matr['suma_provs']  * .6 + matr['suma_clientes'] * .4
matr

,lat,lon,freq_compra,pct_compra_monetario,freq_venta,pct_utilidades,suma_provs,suma_clientes,penal
Aguascalientes,22.000000,-102.500000,0.00,0.00,0.021978,0.010753,0.00,1.011225,0.404490
CDMX,19.320556,-99.151701,0.40,0.35,0.230769,0.258065,0.75,0.972705,0.839082
Chihuahua,28.500000,-106.000000,0.00,0.00,0.054945,0.032258,0.00,1.022687,0.409075
Guadalajara,20.672037,-103.338396,0.25,0.25,0.164835,0.204301,0.50,0.960534,0.684214
Monterrey,25.639784,-100.293102,0.10,0.10,0.120879,0.150538,0.20,0.970341,0.508137
Morelia,19.702712,-101.192382,0.00,0.00,0.065934,0.032258,0.00,1.033676,0.413470
Puebla,18.833333,-98.000000,0.05,0.05,0.098901,0.075269,0.10,1.023632,0.469453
Querétaro,20.854257,-99.847560,0.10,0.05,0.120879,0.118280,0.15,1.002600,0.491040
Saltillo,25.423043,-100.992751,0.00,0.00,0.021978,0.010753,0.00,1.011225,0.404490
Toluca,19.292545,-99.656901,0.00,0.00,0.032967,0.010753,0.00,1.022214,0.408886


### Funcion de distancia. 
En vez de utilizar la distancia Euclideana entre las coordenadas transformadas al sistema mercator, se utiliza directamente la distancia haversine: mide la distancia entre dos puntos sobre la superficie en una esfera. 

In [0]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

### Función objetivo
Función a minimizar. Tiene en cuenta la distancia haversine entre un punto a elegir y cada cliente y proveedor, multiplicado por su penalización. <br/>
Esta penalización es la que tiene en cuenta frecuencia de venta, compra, utilidades y compra monetaria.

In [0]:
def objective_function(x):
  '''
  param x: una lista, cuyo primer valor es longitud, y el segundo es latitud
  '''
  total_cost = [matr['penal'].values[i] * haversine(lon1=x[0], lat1=x[1], lon2=matr['lon'].values[i], lat2=matr['lat'].values[i]) for i in range(matr.shape[0])]
  return np.sum(total_cost)

## Ejecución del optimizador
Se inicializa con la media de latitud y longitud. 

In [0]:

lon_inicial = matr['lon'].mean()
lat_inicial = matr['lat'].mean()
minimize(objective_function, x0=[lon_inicial, lat_inicial])

      fun: 2131.8644462822112
 hess_inv: array([[ 0.00897413, -0.00452849],
       [-0.00452849,  0.00757532]])
      jac: array([0., 0.])
  message: 'Optimization terminated successfully.'
     nfev: 48
      nit: 10
     njev: 12
   status: 0
  success: True
        x: array([-100.40404878,   21.03371073])

In [0]:
# se grafica la ubicación ideal
folium.Marker([21.03371073, -100.40404878], icon=folium.Icon(color='red', icon='info-sign')).add_to(m)
m